In [21]:
import gpflow
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from gpflow.utilities import print_summary, positive

plt.style.use("ggplot")
from gpflow.utilities import ops, print_summary
from gpflow.config import set_default_float, default_float, set_default_summary_fmt
from gpflow.ci_utils import ci_niter
from sklearn.metrics.pairwise import euclidean_distances
set_default_float(np.float64)
set_default_summary_fmt("notebook")
from sklearn.metrics.pairwise import euclidean_distances
%matplotlib inline

In [101]:
# create data
np.random.seed(1001)
N = 300
f = lambda x1, x2: np.sin(12 * x1) + 0.66 * np.cos(25 * x1) + np.sin(12 * x2) + 0.66 * np.cos(25 * x2)
X1 = np.random.rand(N, 1)
X2 = np.random.rand(N, 1)
Y_train = f(X1, X2) + np.random.randn(N, 1) * 0.1 + 3
X_train = np.hstack((X1, X2))
id_train = np.linspace(0,N-1, N).reshape(-1,1)
# create pd matrix for train



# create id of test data
np.random.seed(10)
N = 100
f = lambda x1, x2: np.sin(12 * x1) + 0.66 * np.cos(25 * x1) + np.sin(12 * x2) + 0.66 * np.cos(25 * x2)
X1 = np.random.rand(N, 1)
X2 = np.random.rand(N, 1)
Y_test = f(X1, X2) + np.random.randn(N, 1) * 0.1 + 3
X_test = np.hstack((X1, X2))
id_test = np.linspace(300,390, 100).reshape(-1,1)

# all data
X_all = np.concatenate((X_train, X_test), axis= 0)
Y_all = np.concatenate((Y_train, Y_test), axis= 0)
pd_all = euclidean_distances(X_all, X_all)
np.savetxt('pd_all.txt', pd_all, delimiter =', ') 

In [102]:
class HaversineKernel(gpflow.kernels.SquaredExponential):
    """
    Isotropic RBF Kernel with Haversine distance instead of euclidean distance.
    Assumes 2 dimensional data, with columns [latitude, longitude] in degrees.
    """

    def __init__(self, lengthscales=1.0):
        super().__init__(active_dims=[0])

    def _haversine_dist(self, X, X2):
        pd = np.loadtxt('pd_all.txt', delimiter=",")
        xx = np.unique(np.concatenate((id_test, id_train), axis= 0)).reshape(-1,1)

        len_x = len(xx)
        dist = np.zeros((len_x, len_x))
        for i in range(len_x):
            for j in range(len_x):
                dist[i,j] = pd[int(xx[i]), int(xx[j])]
        
        
#         print(X)
#         pi_on_180 = np.pi / 180

#         f = tf.expand_dims(X * pi_on_180, -2)  # ... x N x 1 x D
#         f2 = tf.expand_dims(X2 * pi_on_180, -3)  # ... x 1 x M x D

#         d = tf.sin((f - f2) / 2) ** 2
#         print(d)
#         lat1, lat2 = tf.expand_dims(X[:, 0] * pi_on_180, -1), tf.expand_dims(X2[:, 0] * pi_on_180, -2)

#         cos_prod = tf.cos(lat2) * tf.cos(lat1)

#         a = d[:, :, 0] + cos_prod * d[:, :, 1]
#         c = tf.asin(tf.sqrt(a)) * 6371 * 2
        return dist

#     @params_as_tensors
    def _scaled_square_dist(self, X, X2):
        """
        Returns (dist(X, X2ᵀ)/lengthscales)².
        """
        if X2 is None:
            X2 = X
        dist = tf.square(self._haversine_dist(X, X2) / self.lengthscales)
        return dist

In [103]:
id_train.reshape(-1,1)

array([[  0.],
       [  1.],
       [  2.],
       [  3.],
       [  4.],
       [  5.],
       [  6.],
       [  7.],
       [  8.],
       [  9.],
       [ 10.],
       [ 11.],
       [ 12.],
       [ 13.],
       [ 14.],
       [ 15.],
       [ 16.],
       [ 17.],
       [ 18.],
       [ 19.],
       [ 20.],
       [ 21.],
       [ 22.],
       [ 23.],
       [ 24.],
       [ 25.],
       [ 26.],
       [ 27.],
       [ 28.],
       [ 29.],
       [ 30.],
       [ 31.],
       [ 32.],
       [ 33.],
       [ 34.],
       [ 35.],
       [ 36.],
       [ 37.],
       [ 38.],
       [ 39.],
       [ 40.],
       [ 41.],
       [ 42.],
       [ 43.],
       [ 44.],
       [ 45.],
       [ 46.],
       [ 47.],
       [ 48.],
       [ 49.],
       [ 50.],
       [ 51.],
       [ 52.],
       [ 53.],
       [ 54.],
       [ 55.],
       [ 56.],
       [ 57.],
       [ 58.],
       [ 59.],
       [ 60.],
       [ 61.],
       [ 62.],
       [ 63.],
       [ 64.],
       [ 65.],
       [ 6

In [104]:
m = gpflow.models.GPR((id_train, Y_train), kernel=k)

# opt = gpflow.optimizers.Scipy()
# opt.minimize(m.training_loss, variables=m.trainable_variables)


mean, var = m.predict_y(id_test)
# plot(m, title="trained model")

In [106]:
var

<tf.Tensor: shape=(100, 1), dtype=float64, numpy=
array([[1.08827336],
       [1.09532161],
       [1.10291035],
       [1.11106604],
       [1.11981544],
       [1.12918569],
       [1.13920417],
       [1.14989851],
       [1.16129649],
       [1.17342604],
       [1.18631512],
       [1.19999174],
       [1.21448382],
       [1.2298192 ],
       [1.24602551],
       [1.26313018],
       [1.28116033],
       [1.30014271],
       [1.32010366],
       [1.341069  ],
       [1.36306402],
       [1.38611337],
       [1.41024103],
       [1.43547022],
       [1.46182334],
       [1.48932191],
       [1.51798652],
       [1.54783677],
       [1.57889117],
       [1.61116714],
       [1.64468091],
       [1.6794475 ],
       [1.71548065],
       [1.75279274],
       [1.79139483],
       [1.83129652],
       [1.87250597],
       [1.91502983],
       [1.95887323],
       [2.00403973],
       [2.05053129],
       [2.09834826],
       [2.14748933],
       [2.19795156],
       [2.24973031],
     